In [42]:
import re
import cv2
import numpy as np
import torch
from PIL import Image
import warnings
from transformers import AutoTokenizer, AutoModel
import easyocr
import fitz  # PyMuPDF
import os
import pickle


warnings.filterwarnings('ignore')
        
class ChildcareDocumentClassifier:
    def __init__(self, similarity_threshold: float = 0.70):
        print("📥 모델 로딩 중...")
        print(" - MPNet 임베딩...")
        self.mpnet_tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/all-mpnet-base-v2")
        self.mpnet_model = AutoModel.from_pretrained("sentence-transformers/all-mpnet-base-v2")
        self.mpnet_model.eval()
        
        print(" - EasyOCR (이미지)...")
        self.easyocr_reader = easyocr.Reader(["ko", "en"], gpu=False)

        # ✅ 기존 변수들 그대로 유지!
        self.KEYWORDS = ["보육교사", "보육", "영유아보육법", "보육교사자격증", "유아교육법"]
        self.REFERENCES = [
            "보육교사", "영유아보육법에 따라 보육교사의", "보육교사 1급", "보육교사 2급", "보육교사 3급", "영유아보육법", "유아교육법"
        ]
        
        self.threshold = similarity_threshold
        print("✅ 모든 모델 로딩 완료!")

    def extract_text_mpnet_chandra(self, file_path: str) -> str:
        """PyMuPDF + EasyOCR"""
        try:
            ext = os.path.splitext(file_path)[1].lower()
            
            # 이미지 파일인 경우 → EasyOCR
            if ext in ['.png', '.jpg', '.jpeg', '.tiff', '.bmp', '.webp']:
                print("🖼️  이미지 → EasyOCR")
                image = Image.open(file_path).convert("RGB")
                result = self.easyocr_reader.readtext(np.array(image))
                text = " ".join([txt for _, txt, conf in result if conf > 0.3])
                text = re.sub(r"\s+", " ", text).strip()
                print(f"✅ EasyOCR: {len(text)}자")
                return text
            
            # PDF 파일인 경우 → PyMuPDF 텍스트 우선
            elif ext == '.pdf':
                print("📄 PDF → PyMuPDF 처리...")
                doc = fitz.open(file_path)
                
                # 1. 텍스트 레이어 추출 (첫 3페이지)
                text_chunks = []
                for page_num in range(min(3, len(doc))):
                    page = doc[page_num]
                    text = page.get_text()
                    if text.strip():
                        text_chunks.append(text)
                
                full_text = re.sub(r"\s+", " ", " ".join(text_chunks)).strip()
                print(f"✅ PyMuPDF 텍스트: {len(full_text)}자")
                
                # 텍스트 충분하면 바로 반환
                if len(full_text) > 30:
                    doc.close()
                    return full_text
                
                # 2. 스캔 PDF일 경우 → PyMuPDF로 이미지 렌더링 + EasyOCR
                print("📄 스캔 PDF 감지 → PyMuPDF 렌더링 + EasyOCR...")
                ocr_chunks = []
                for page_num in range(min(3, len(doc))):  # 첫 3페이지만
                    page = doc[page_num]
                    pix = page.get_pixmap(dpi=150)
                    img_data = Image.frombytes("RGB", [pix.width, pix.height], pix.samples)
                    
                    # EasyOCR 적용
                    ocr_result = self.easyocr_reader.readtext(np.array(img_data))
                    page_ocr = " ".join([txt for _, txt, conf in ocr_result if conf > 0.3])
                    if page_ocr.strip():
                        ocr_chunks.append(page_ocr)
                
                ocr_text = re.sub(r"\s+", " ", " ".join(ocr_chunks)).strip()
                doc.close()
                print(f"✅ PyMuPDF+EasyOCR: {len(ocr_text)}자")
                return ocr_text if ocr_text else os.path.splitext(os.path.basename(file_path))[0]
            
            else:
                print(f"❌ 미지원 형식: {ext}")
                return os.path.splitext(os.path.basename(file_path))[0]
                
        except Exception as e:
            print(f"❌ 처리 실패: {e}")
            return os.path.splitext(os.path.basename(file_path))[0]


    def mpnet_encode(self, sentences: list) -> torch.Tensor:
        if len(sentences) == 0:
            return torch.empty((0, 768))
        encoded_input = self.mpnet_tokenizer(sentences, padding=True, truncation=True, max_length=384, return_tensors="pt")
        with torch.no_grad():
            model_output = self.mpnet_model(**encoded_input)
            embeddings = model_output.last_hidden_state[:, 0, :]
        return torch.nn.functional.normalize(embeddings, p=2, dim=1)

    def preprocess_sentences(self, text: str) -> list:
        text = re.sub(r"[^\w\s가-힣.!?0-9]", " ", text)
        text = re.sub(r"\s+", " ", text).strip()
        text = re.sub(r"(습니다\.|입니다\.|다\.|요\.)", r"\1<SPLIT>", text)
        text = re.sub(r"([\.!?])", r"\1<SPLIT>", text)
        sentences = text.split("<SPLIT>")
        return [s.strip() for s in sentences if len(s.strip()) > 5]

    def compute_mpnet_similarity(self, ref_embedding: torch.Tensor, doc_embeddings: torch.Tensor) -> tuple:
        if doc_embeddings.numel() == 0:
            return 0.0, 0
        similarities = torch.nn.functional.cosine_similarity(ref_embedding.unsqueeze(0), doc_embeddings, dim=1)
        return float(similarities.max()), int(similarities.argmax())

    def classify_childcare_document(self, file_path: str, reference_sentences: list) -> dict:
        result = {
            "verdict": False, "max_confidence": 0.0, "matched_sentence": "",
            "total_sentences": 0, "raw_text": "", "reason": ""
        }
        
        raw_text = self.extract_text_mpnet_chandra(file_path)
        result["raw_text"] = raw_text
        
        doc_sentences = self.preprocess_sentences(raw_text)
        result["total_sentences"] = len(doc_sentences)
        
        if len(doc_sentences) == 0:
            result["reason"] = "문장 없음"
            return result

        all_sentences = reference_sentences + doc_sentences
        all_embeddings = self.mpnet_encode(all_sentences)
        
        ref_embeddings = all_embeddings[:len(reference_sentences)]
        doc_embeddings = all_embeddings[len(reference_sentences):]

        best_matches = []
        for i, ref_emb in enumerate(ref_embeddings):
            score, doc_idx = self.compute_mpnet_similarity(ref_emb, doc_embeddings)
            best_matches.append({
                "ref_idx": i, "score": score, "matched_sentence": doc_sentences[doc_idx]
            })

        max_match = max(best_matches, key=lambda x: x["score"])
        result["max_confidence"] = round(max_match["score"], 4)
        result["matched_sentence"] = max_match["matched_sentence"]
        result["verdict"] = max_match["score"] >= self.threshold
        
        return result
    def classify_childcare_with_rules(self, file_path: str, base_threshold=0.70):
        """클래스 메서드로 변경 - self가 첫 번째 인자"""
        result = self.classify_childcare_document(file_path, self.REFERENCES)
        score = result.get("max_confidence", 0.0)
        raw_text = result.get("raw_text", "")
        
        # KEYWORDS를 self.KEYWORDS로 참조
        hit_count = sum(raw_text.count(kw) for kw in self.KEYWORDS)
        score += 0.02 * hit_count if hit_count > 0 else -0.05
        
        result["rule_score"] = round(score, 4)
        result["verdict"] = score >= base_threshold
        result["keyword_hit_count"] = hit_count
        return result
    
    def save_model(self, save_path: str = "childcare_model.pkl"):
        """🔥 모델 전체 저장 (pickle)"""
        with open(save_path, 'wb') as f:
            pickle.dump(self, f)
        
        file_size = os.path.getsize(save_path) / 1024 / 1024
        print(f"✅ 모델 저장 완료: {save_path} ({file_size:.1f}MB)")
    
    @classmethod
    def load_model(cls, load_path: str):
        """📥 모델 전체 로드 (pickle)"""
        print(f"📥 모델 로딩: {load_path}")
        
        with open(load_path, 'rb') as f:
            classifier = pickle.load(f)
        
        print("✅ 모델 로드 완료!")
        return classifier


if __name__ == "__main__":
    
    classifier = ChildcareDocumentClassifier(0.70)
    
    # ✅ 단일 path로 이미지/PDF 모두 처리!
    test_path = r"C:\Users\ehjun\Documents\캡스톤디자인\자동문서인식모델\data\보육자격증 아닌것\토익성적.png"  # 이미지
    # test_path = r"C:\Users\ehjun\Documents\...(프로젝트신청서.pdf"  # PDF
    
    result = classifier.classify_childcare_with_rules(test_path)
    
    print("🎯 최종 판정:", "보육 ✅" if result["verdict"] else "일반 ❌")
    print(f"📊 similarity: {result['max_confidence']}")
    print(f"📈 rule_score: {result['rule_score']}")
    print(f"🔍 키워드: {result['keyword_hit_count']}")
    print(f"📄 텍스트: {result['raw_text'][:100]}...")


📥 모델 로딩 중...
 - MPNet 임베딩...


Using CPU. Note: This module is much faster with a GPU.


 - EasyOCR (이미지)...
✅ 모든 모델 로딩 완료!
🖼️  이미지 → EasyOCR
✅ EasyOCR: 336자
🎯 최종 판정: 일반 ❌
📊 similarity: 0.4827
📈 rule_score: 0.4327
🔍 키워드: 0
📄 텍스트: 문서확인번호 : 1761-2836-1342-6760 진 본 2025/10/24 14,26.55 제 2025- 00205325호 어학성적 사전등록 확인서 성 명: 박 여 경 생 년 ...


In [40]:
# ===== 셀 2: 테스트 =====
test_path = r"C:\Users\ehjun\Documents\캡스톤디자인\자동문서인식모델\data\보육자격증 아닌것\토익성적.png"

result = classifier.classify_childcare_with_rules(test_path)

print("🎯 최종 판정:", "보육 ✅" if result["verdict"] else "일반 ❌")
print(f"📊 similarity: {result['max_confidence']}")
print(f"📈 rule_score: {result['rule_score']}")
print(f"🔍 키워드: {result['keyword_hit_count']}")


🖼️  이미지 → EasyOCR
✅ EasyOCR: 336자
🎯 최종 판정: 일반 ❌
📊 similarity: 0.4827
📈 rule_score: 0.4327
🔍 키워드: 0


In [43]:
# ===== 셀 3: 모델 저장 =====
classifier.save_model("childcare_model.pkl")


✅ 모델 저장 완료: childcare_model.pkl (505.5MB)


In [ ]:
# ===== 셀 4: 모델 로드=====
loaded_classifier = ChildcareDocumentClassifier.load_model("childcare_model.pkl")
print("✅ 로드 완료! 바로 사용 가능")


📥 모델 로딩: childcare_model.pkl
✅ 모델 로드 완료!
✅ 로드 완료! 바로 사용 가능


In [46]:
# ===== 셀 5: 로드된 모델로 테스트 =====
test_path2 = r"C:\Users\ehjun\Documents\캡스톤디자인\자동문서인식모델\data\pdf\보육자격증\보육교사_자격증.pdf.pdf"

result2 = loaded_classifier.classify_childcare_with_rules(test_path2)
print("🎯 판정:", "보육 ✅" if result2["verdict"] else "일반 ❌")


📄 PDF → PyMuPDF 처리...
✅ PyMuPDF 텍스트: 0자
📄 스캔 PDF 감지 → PyMuPDF 렌더링 + EasyOCR...
✅ PyMuPDF+EasyOCR: 145자
🎯 판정: 보육 ✅
